In [21]:
from re import X
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn import preprocessing, model_selection
import keras
import numpy as np

In [22]:
url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.KerasLayer(url)

In [23]:
data = pd.read_csv("transcript_data.csv", encoding="latin-1")

In [24]:
y = list(data['v1'])
x = list(data['v2'])

In [25]:
le = preprocessing.LabelEncoder()
le.fit(y)

LabelEncoder()

In [26]:
def encode(le, labels):
  enc = le.transform(labels)
  return keras.utils.to_categorical(enc)

def decode(le, one_hot):
  dec = np.argmax(one_hot, axis=1)
  return le.inverse_transform(dec)

In [27]:
test = encode(le, ['class_name','other','other','class_name'])
test

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.]], dtype=float32)

In [28]:
untest = decode(le, test)
untest

array(['class_name', 'other', 'other', 'class_name'], dtype='<U10')

#***To be clear, we will need to increase the size of this dataset***

currently each cell value represents one line read from the OCR in the csv

In [39]:
x_enc = x
y_enc = encode(le, y)

x_train = np.asarray(x_enc[:-20])
y_train = np.asarray(y_enc[:-20])

x_test = np.asarray(x_enc[-20:])
y_test = np.asarray(y_enc[-20:])

In [40]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

In [41]:
# converting string words into numerical vectors
def UniversalEmbedding(x):
  return embed(tf.squeeze(tf.cast(x, tf.string)))

In [42]:
input_text = Input(shape=(1,),dtype=tf.string)
embedding = Lambda(UniversalEmbedding,output_shape=(512,))(input_text)
dense = Dense(256,activation='relu')(embedding)
pred = Dense(3,activation='softmax')(dense)
model = Model(inputs=[input_text],outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [43]:
history = model.fit(x_train, y_train, epochs=3, batch_size=32)
model.save_weights('./model.h5')

Epoch 1/3
5/5 [==============================] - 3s 282ms/step - loss: 1.0544 - accuracy: 0.6026
Epoch 2/3
5/5 [==============================] - 1s 253ms/step - loss: 0.8758 - accuracy: 0.8411
Epoch 3/3
5/5 [==============================] - 1s 280ms/step - loss: 0.7324 - accuracy: 0.8477


In [44]:
model.load_weights('./model.h5')
preds = model.predict(x_test,batch_size=32)

1/1 [==============================] - 1s 1s/step


In [45]:
y_test = decode(le, y_test)
y_preds = decode(le, preds)
y_preds

array(['No_value', 'No_value', 'No_value', 'No_value', 'No_value',
       'No_value', 'No_value', 'No_value', 'class_name', 'No_value',
       'No_value', 'No_value', 'No_value', 'No_value', 'class_name',
       'No_value', 'No_value', 'class_name', 'No_value', 'No_value'],
      dtype='<U10')

In [47]:
from sklearn import metrics
metrics.confusion_matrix(y_test,y_preds)
print(metrics.classification_report(y_test,y_preds))

              precision    recall  f1-score   support

    No_value       0.76      0.93      0.84        14
  class_name       0.33      1.00      0.50         1
       other       0.00      0.00      0.00         5

    accuracy                           0.70        20
   macro avg       0.37      0.64      0.45        20
weighted avg       0.55      0.70      0.61        20



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
